In [ ]:
import hvplot.pandas
import holoviews as hv, pandas as pd, colorcet as cc
from holoviews.element.tiles import EsriImagery
from holoviews.operation.datashader import datashade
hv.extension('bokeh')

# Read in data

In [ ]:
df = pd.read_parquet(
    'https://s3.amazonaws.com/datashader-data/nyc_taxi_wide.parq',
    columns = ['dropoff_x', 'dropoff_y']
)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
%%time
# it's so slow and plot looks terrible when I use pandas default .plot
df.plot.scatter('dropoff_x', 'dropoff_y')

# Plotting

In [ ]:
%%time
map_tiles = EsriImagery().opts(alpha=0.5, width=900, height=480, bgcolor='black')
points = hv.Points(df, ['dropoff_x', 'dropoff_y'])
taxi_trips = datashade(points, cmap=cc.fire, width=900, height=480)
map_tiles * taxi_trips

In [ ]:
map_tiles = EsriImagery().opts(alpha=0.5, width=900, height=480, bgcolor='black')
plot = df.hvplot(
    'dropoff_x',
    'dropoff_y',
    kind='scatter',
    rasterize=True,
    cmap=cc.fire,
    cnorm='eq_hist'
)
map_tiles * plot

# How does Datashader work? 


In [ ]:
import datashader as ds
import datashader.transfer_functions as tf
from holoviews.operation.datashader import rasterize

## Projection

In [ ]:
canvas = ds.Canvas(plot_width=900, plot_height=480)

In [ ]:
canvas

## Aggregation

In [ ]:
canvas.points(df, 'dropoff_x', 'dropoff_y', agg=ds.count())

## Transformation (optional)

In [ ]:
import numpy as np

In [ ]:
np.power(canvas.points(df, 'dropoff_x', 'dropoff_y', agg=ds.count()),2)

## Colormapping

In [ ]:
tf.set_background(
    tf.shade(
        canvas.points(df, 'dropoff_x', 'dropoff_y', agg=ds.count()),
        cmap=cc.fire
    ),
    'black'
)

## Embedding

In [ ]:
map_tiles = EsriImagery().opts(alpha=0.5, width=900, height=480, bgcolor='black')
points = hv.Points(df, ['dropoff_x', 'dropoff_y'])
ropts = dict(tools=['hover'], colorbar=True, colorbar_position='bottom', cmap=cc.fire, cnorm='eq_hist')
taxi_trips = rasterize(points).opts(**ropts)
map_tiles * taxi_trips